In [79]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port','0'). \
config('spark.shuffle.useOldFetchProtocol','true'). \
config("spark.sql.warehouse.dir", f"/user/itv007136/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [80]:
spark

# Associating points to grades in order to calculate the Loan Score

In [81]:
spark.conf.set("spark.sql.unacceptable_rated_pts",0)
spark.conf.set("spark.sql.very_bad_rated_pts",100)
spark.conf.set("spark.sql.bad_rated_pts",250)
spark.conf.set("spark.sql.good_rated_pts",500)
spark.conf.set("spark.sql.very_good_rated_pts",650)
spark.conf.set("spark.sql.excellent_rated_pts",800)

In [82]:
spark.conf.set("spark.sql.unacceptable_grade_pts",750)
spark.conf.set("spark.sql.very_bad_grade_pts",1000)
spark.conf.set("spark.sql.bad_grade_pts",1500)
spark.conf.set("spark.sql.good_grade_pts",2000)
spark.conf.set("spark.sql.very_good_grade_pts",2500)

# The tables that are required to calculate the loan score are:

customers_new

loans

loans_repayments

loans_defaulters_delinq_new

loans_defaulters_detail_rec_enq_new

# Loan Score Calculation Criteria 1: Payment History(ph)

In [83]:
 bad_customer_data_final_df= spark.read \
    .format("csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load("/public/trendytech/lendingclubproject/bad/bad_customer_data_final")

In [84]:
bad_customer_data_final_df.createOrReplaceTempView("bad_data_customer")

In [85]:
ph_df=spark.sql("""
    Select member_id,
    CASE
         WHEN r.last_payment_amount < (l.monthly_installment * 0.5) THEN ${spark.sql.very_bad_rated_pts}
         WHEN r.last_payment_amount >= (l.monthly_installment * 0.5) AND r.last_payment_amount< l.monthly_installment THEN ${spark.sql.bad_rated_pts}
         WHEN r.last_payment_amount = l.monthly_installment THEN ${spark.sql.good_rated_pts}
         WHEN r.last_payment_amount > l.monthly_installment AND r.last_payment_amount <= (l.monthly_installment * 1.5) THEN ${spark.sql.very_good_rated_pts}
         WHEN r.last_payment_amount > (l.monthly_installment * 1.5) THEN ${spark.sql.excellent_rated_pts}
         ELSE ${spark.sql.unacceptable_rated_pts}
    END as last_payment_pts,
    CASE 
         WHEN r.total_payment_received >= (l.funded_amount * 0.5) THEN ${spark.sql.very_good_rated_pts}
         WHEN r.total_payment_received < (l.funded_amount * 0.5)  AND r.total_payment_received>0 THEN ${spark.sql.good_rated_pts}
         WHEN r.total_payment_received = 0 OR r.total_payment_received IS NULL THEN ${spark.sql.unacceptable_rated_pts}
    END as total_payment_pts
    from itv007136_lending_club.loans_repayments r 
    inner join itv007136_lending_club.loans l
    on r.loan_id=l.loan_id 
    where l.member_id not in (select member_id from bad_data_customer)
""")

In [86]:
ph_df.createOrReplaceTempView("ph_pts")

# Loan Score Calculation Criteria 2: Loan Defaulters History (ldh)

In [87]:
ldh_ph_df= spark.sql("""
    SELECT p.*,
    CASE 
    WHEN d.delinq_2yrs = 0 THEN ${spark.sql.excellent_rated_pts}
    WHEN d.delinq_2yrs BETWEEN 1 AND 2 THEN ${spark.sql.bad_rated_pts}
    WHEN d.delinq_2yrs BETWEEN 3 AND 5 THEN ${spark.sql.very_bad_rated_pts}
    WHEN d.delinq_2yrs > 5  OR d.delinq_2yrs IS NULL THEN ${spark.sql.unacceptable_rated_pts}
    END AS delinq_pts,
    CASE 
    WHEN r.pub_rec = 0 THEN ${spark.sql.excellent_rated_pts}
    WHEN r.pub_rec BETWEEN 1 AND 2 THEN ${spark.sql.bad_rated_pts}
    WHEN r.pub_rec BETWEEN 3 AND 5 THEN ${spark.sql.very_bad_rated_pts}
    WHEN r.pub_rec > 5  OR r.pub_rec IS NULL THEN ${spark.sql.unacceptable_rated_pts}
    END AS public_record_pts,
    CASE 
    WHEN r.pub_rec_bankruptcies = 0 THEN ${spark.sql.excellent_rated_pts}
    WHEN r.pub_rec_bankruptcies BETWEEN 1 AND 2 THEN ${spark.sql.bad_rated_pts}
    WHEN r.pub_rec_bankruptcies BETWEEN 3 AND 5 THEN ${spark.sql.very_bad_rated_pts}
    WHEN r.pub_rec_bankruptcies > 5  OR r.pub_rec_bankruptcies IS NULL THEN ${spark.sql.unacceptable_rated_pts}
    END AS public_bankruptcies_pts,
    CASE 
    WHEN r.inq_last_6mths = 0 THEN ${spark.sql.excellent_rated_pts}
    WHEN r.inq_last_6mths BETWEEN 1 AND 2 THEN ${spark.sql.bad_rated_pts}
    WHEN r.inq_last_6mths BETWEEN 3 AND 5 THEN ${spark.sql.very_bad_rated_pts}
    WHEN r.inq_last_6mths > 5  OR r.inq_last_6mths IS NULL THEN ${spark.sql.unacceptable_rated_pts}
    END AS enq_pts
    FROM itv007136_lending_club.loans_defaulters_delinq d
    inner join itv007136_lending_club.loans_defaulters_detail_rec_enq r on d.member_id = r.member_id
    inner join ph_pts p on r.member_id=p.member_id
    where r.member_id not in (select member_id from bad_data_customer)
    """)

In [88]:
ldh_ph_df.createOrReplaceTempView("ldh_ph_pts")

# Criteria 3: Financial Health(fh)

In [89]:
fh_ldh_ph_df = spark.sql("select ldef.*, \
   CASE \
   WHEN LOWER(l.loan_status) LIKE '%fully paid%' THEN ${spark.sql.excellent_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%current%' THEN ${spark.sql.good_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%in grace period%' THEN ${spark.sql.bad_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%late (16-30 days)%' OR LOWER(l.loan_status) LIKE '%late (31-120 days)%' THEN ${spark.sql.very_bad_rated_pts} \
   WHEN LOWER(l.loan_status) LIKE '%charged off%' THEN ${spark.sql.unacceptable_rated_pts} \
   else ${spark.sql.unacceptable_rated_pts} \
   END AS loan_status_pts, \
   CASE \
   WHEN LOWER(a.home_ownership) LIKE '%own' THEN ${spark.sql.excellent_rated_pts} \
   WHEN LOWER(a.home_ownership) LIKE '%rent' THEN ${spark.sql.good_rated_pts} \
   WHEN LOWER(a.home_ownership) LIKE '%mortgage' THEN ${spark.sql.bad_rated_pts} \
   WHEN LOWER(a.home_ownership) LIKE '%any' OR LOWER(a.home_ownership) IS NULL THEN ${spark.sql.very_bad_rated_pts} \
   END AS home_pts, \
   CASE \
   WHEN l.funded_amount <= (a.total_high_credit_limit * 0.10) THEN ${spark.sql.excellent_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.10) AND l.funded_amount <= (a.total_high_credit_limit * 0.20) THEN ${spark.sql.very_good_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.20) AND l.funded_amount <= (a.total_high_credit_limit * 0.30) THEN ${spark.sql.good_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.30) AND l.funded_amount <= (a.total_high_credit_limit * 0.50) THEN ${spark.sql.bad_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.50) AND l.funded_amount <= (a.total_high_credit_limit * 0.70) THEN ${spark.sql.very_bad_rated_pts} \
   WHEN l.funded_amount > (a.total_high_credit_limit * 0.70) THEN ${spark.sql.unacceptable_rated_pts} \
   else ${spark.sql.unacceptable_rated_pts} \
   END AS credit_limit_pts, \
   CASE \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A1' THEN ${spark.sql.excellent_rated_pts} \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A2' THEN (${spark.sql.excellent_rated_pts} * 0.95) \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A3' THEN (${spark.sql.excellent_rated_pts} * 0.90) \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A4' THEN (${spark.sql.excellent_rated_pts} * 0.85) \
   WHEN (a.grade) = 'A' and (a.sub_grade)='A5' THEN (${spark.sql.excellent_rated_pts} * 0.80) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B1' THEN (${spark.sql.very_good_rated_pts}) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B2' THEN (${spark.sql.very_good_rated_pts} * 0.95) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B3' THEN (${spark.sql.very_good_rated_pts} * 0.90) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B4' THEN (${spark.sql.very_good_rated_pts} * 0.85) \
   WHEN (a.grade) = 'B' and (a.sub_grade)='B5' THEN (${spark.sql.very_good_rated_pts} * 0.80) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C1' THEN (${spark.sql.good_rated_pts}) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C2' THEN (${spark.sql.good_rated_pts} * 0.95) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C3' THEN (${spark.sql.good_rated_pts} * 0.90) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C4' THEN (${spark.sql.good_rated_pts} * 0.85) \
   WHEN (a.grade) = 'C' and (a.sub_grade)='C5' THEN (${spark.sql.good_rated_pts} * 0.80) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D1' THEN (${spark.sql.bad_rated_pts}) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D2' THEN (${spark.sql.bad_rated_pts} * 0.95) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D3' THEN (${spark.sql.bad_rated_pts} * 0.90) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D4' THEN (${spark.sql.bad_rated_pts} * 0.85) \
   WHEN (a.grade) = 'D' and (a.sub_grade)='D5' THEN (${spark.sql.bad_rated_pts} * 0.80) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E1' THEN (${spark.sql.very_bad_rated_pts}) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E2' THEN (${spark.sql.very_bad_rated_pts} * 0.95) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E3' THEN (${spark.sql.very_bad_rated_pts} * 0.90) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E4' THEN (${spark.sql.very_bad_rated_pts} * 0.85) \
   WHEN (a.grade) = 'E' and (a.sub_grade)='E5' THEN (${spark.sql.very_bad_rated_pts} * 0.80) \
   WHEN (a.grade) in ('F', 'G') THEN (${spark.sql.unacceptable_rated_pts}) \
   END AS grade_pts \
   FROM ldh_ph_pts ldef \
   INNER JOIN itv007136_lending_club.loans l ON ldef.member_id = l.member_id \
   INNER JOIN itv007136_lending_club.customers_new a ON a.member_id = ldef.member_id where ldef.member_id NOT IN (select member_id from bad_data_customer)") 

In [90]:
fh_ldh_ph_df.createOrReplaceTempView("fh_ldh_ph_pts")

In [91]:
fh_ldh_ph_df

member_id,last_payment_pts,total_payment_pts,delinq_pts,public_record_pts,public_bankruptcies_pts,enq_pts,loan_status_pts,home_pts,credit_limit_pts,grade_pts
000c8875b71a6b47c...,800,650,250,800,800,800,800,250,800,680.00
003769d7f54c7859e...,500,500,250,800,800,800,0,250,800,100.00
003e1e6cbd2920bbb...,500,650,250,250,250,800,500,250,800,640.00
004017b21bd4d6271...,100,650,0,800,800,800,800,250,800,500.00
005b4c3db3fce07dc...,500,650,250,250,800,250,500,250,500,520.00
00710707c563c2119...,800,650,250,800,800,800,800,250,800,520.00
007da79904f69970d...,800,650,250,800,800,800,800,500,250,800.00
00f435a80d0440ece...,500,500,100,800,800,800,500,250,800,475.00
00fc2ae3ffb1213e4...,650,650,250,800,800,800,800,250,650,450.00
00fc8144cb210ba8c...,500,650,250,250,250,800,500,250,800,500.00


#Final Loan Score calculation by consdering all the 3 criterias with the following %**

    1. Payment History = 20%
    2. Loan Defaults = 45%
    3. Financial Health = 35%

In [92]:
loan_score = spark.sql("SELECT member_id, \
((last_payment_pts+total_payment_pts)*0.20) as payment_history_pts, \
((delinq_pts + public_record_pts + public_bankruptcies_pts + enq_pts) * 0.45) as defaulters_history_pts, \
((loan_status_pts + home_pts + credit_limit_pts + grade_pts)*0.35) as financial_health_pts \
FROM fh_ldh_ph_pts")

In [93]:
from pyspark.sql.functions import expr

In [94]:
final_loan_score=loan_score.withColumn("loan_score",expr("payment_history_pts + defaulters_history_pts + financial_health_pts"))

In [95]:
final_loan_score

member_id,payment_history_pts,defaulters_history_pts,financial_health_pts,loan_score
000c8875b71a6b47c...,290.00,1192.50,885.5000,2368.0000
003769d7f54c7859e...,200.00,1192.50,402.5000,1795.0000
003e1e6cbd2920bbb...,230.00,697.50,766.5000,1694.0000
004017b21bd4d6271...,150.00,1080.00,822.5000,2052.5000
005b4c3db3fce07dc...,230.00,697.50,619.5000,1547.0000
00710707c563c2119...,290.00,1192.50,829.5000,2312.0000
007da79904f69970d...,290.00,1192.50,822.5000,2305.0000
00f435a80d0440ece...,200.00,1125.00,708.7500,2033.7500
00fc2ae3ffb1213e4...,260.00,1192.50,752.5000,2205.0000
00fc8144cb210ba8c...,230.00,697.50,717.5000,1645.0000


In [96]:
final_loan_score.createOrReplaceTempView("loan_score_eval")

In [97]:
loan_score_final = spark.sql("select ls.*, \
case \
WHEN loan_score > ${spark.sql.very_good_grade_pts} THEN 'A' \
WHEN loan_score <= ${spark.sql.very_good_grade_pts} AND loan_score > ${spark.sql.good_grade_pts} THEN 'B' \
WHEN loan_score <= ${spark.sql.good_grade_pts} AND loan_score > ${spark.sql.bad_grade_pts} THEN 'C' \
WHEN loan_score <= ${spark.sql.bad_grade_pts} AND loan_score  > ${spark.sql.very_bad_grade_pts} THEN 'D' \
WHEN loan_score <= ${spark.sql.very_bad_grade_pts} AND loan_score > ${spark.sql.unacceptable_grade_pts} THEN 'E'  \
WHEN loan_score <= ${spark.sql.unacceptable_grade_pts} THEN 'F' \
end as loan_final_grade \
from loan_score_eval ls")

In [98]:
loan_score_final.createOrReplaceTempView("loan_final_table")

In [100]:
loan_score_final.count()

482842

In [101]:
loan_score_final.write \
.format("parquet") \
.mode("overwrite") \
.option("path","/user/itv007136/lendingclubproject/processed/loan_score") \
.save()

In [102]:
spark.stop()